In [1]:
import os
from os import path
import pandas as pd

### Build results dataframe

In [4]:
N = 24
# tls = [0.5, 0.6, 0.7]
lls = [0.0, 0.1, 0.2]
tls = [1.0]
seeds = [i for i in range(1, 4)]
steps = [(i + 1) * 5000 for i in range(N)]

In [7]:
results_list_tl = []
performance_path_tl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/models_learned_cost/planning_results'
name_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal={ll}' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=True-lambdatl={tl}' + \
       '-seed={seed}-novaluestep{step}.model.log'
for ll in lls:
    for tl in tls:
        for seed in seeds:
            for step in steps:
                file_name = path.join(performance_path_tl, name_tl.format(ll=ll, tl=tl, seed=seed, step=step))
                if os.path.isfile(file_name):
                    with open(file_name) as f:
                        lines = f.readlines()
                        for line in lines:
                            splits = line.split("|")
                            ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                            time = int(splits[1].split(":")[1].strip())
                            distance = int(splits[2].split(":")[1].strip())
                            success_or_failure = int(splits[3].split(":")[1].strip())
                            df_line = {
                                'lambda_l': ll,
                                'lambda_tl': tl,
                                'seed': seed,
                                'step': step,
                                'ep': ep,
                                'time': time,
                                'distance': distance,
                                'result': success_or_failure
                            }
                            results_list_tl.append(df_line)

In [8]:
results_df_tl = pd.DataFrame(results_list_tl)
cols_tl = ['lambda_l', 'lambda_tl', 'seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_tl = results_df_tl[cols_tl]

In [9]:
results_list_bl = []
performance_path_bl = '{/misc/vlgscratch4/LecunGroup/}nvidia-collab/yairschiff/pytorch-PPUU/' + \
                      'models_baseline/planning_results'
name_bl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=0-learnedcost=False' + \
       '-seed={seed}-novaluestep{step}.model.log'
for seed in seeds:
    for step in steps:
        file_name = path.join(performance_path_bl, name_bl.format(seed=seed, step=step))
        if os.path.isfile(file_name):
            with open(file_name) as f:
                lines = f.readlines()
                for line in lines:
                    splits = line.split("|")
                    ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                    time = int(splits[1].split(":")[1].strip())
                    distance = int(splits[2].split(":")[1].strip())
                    success_or_failure = int(splits[3].split(":")[1].strip())
                    df_line = {
                        'seed': seed,
                        'step': step,
                        'ep': ep,
                        'time': time,
                        'distance': distance,
                        'result': success_or_failure
                    }
                    results_list_bl.append(df_line)

In [10]:
results_df_bl = pd.DataFrame(results_list_bl)
cols_bl = ['seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_bl = results_df_bl[cols_bl]

KeyError: "None of [Index(['seed', 'step', 'ep', 'time', 'distance', 'result'], dtype='object')] are in the [columns]"

### Filter dataframe

In [12]:
# Set filter variables for target_lane results
ll = 0.0
tl = 1.0
seed = 1
step = 25000
result = 0  # 1 for successes, 0 for failures

In [17]:
# Perform filter on target_lane results
print(f'List of {"success" if result else "failure"}' +
      f'episodes for lambda_l: {ll}, lambda_tl: {tl}, seed: {seed}, step: {step}:')
filtered_list_tl = results_df_tl[(results_df_tl.lambda_l == ll) &
                                 (results_df_tl.lambda_tl == tl) &
                                 (results_df_tl.seed == seed) &
                                 (results_df_tl.step == step) &
                                 (results_df_tl.result == result)]['ep'].values
print(len(filtered_list_tl))
print(filtered_list_tl)

List of failureepisodes for lambda_l: 0.0, lambda_tl: 1.0, seed: 1, step: 25000:
190
[  1   2   5   6   9  10  12  14  15  17  19  20  21  22  23  25  31  33
  34  39  41  43  44  46  50  51  55  59  60  71  73  77  81  82  83  91
  93 118 122 123 124 125 131 135 136 143 146 151 159 164 166 167 170 172
 175 179 180 181 188 189 192 203 206 208 211 214 217 219 224 225 226 227
 229 232 233 234 235 237 239 240 248 252 255 256 262 264 271 276 280 282
 283 285 288 290 298 300 305 306 307 309 315 320 323 324 325 329 333 335
 336 345 348 350 352 353 358 361 363 366 367 370 374 384 385 386 390 391
 392 394 404 405 407 409 410 412 413 414 417 418 420 423 424 433 436 441
 448 450 452 454 456 459 461 462 464 465 468 471 472 473 476 480 483 488
 490 496 501 503 504 505 509 511 513 515 516 519 522 524 525 526 528 530
 532 533 534 538 543 547 550 554 556 561]


In [ ]:
# Set filter variables for baseline results
seed = 1
step = 115000
result = 0  # 1 for successes, 0 for failures

In [ ]:
# Perform filter on baseline results
print(f'List of {"success" if result else "failure"} episodes for baseline - seed: {seed}, step: {step}:')
filtered_list_bl = results_df_bl[(results_df_bl.seed == seed) &
                                 (results_df_bl.step == step) &
                                 (results_df_bl.result == result)]['ep'].values
print(len(filtered_list_bl))
print(filtered_list_bl)